In [1]:
from transformers import pipeline
import pandas as pd
import numpy as np

In [2]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

config.json: 0.00B [00:00, ?B/s]

C:\Users\wills\dev\Trojan-Parse-Project\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wills\.cache\huggingface\hub\models--MoritzLaurer--DeBERTa-v3-base-mnli-fever-anli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cpu


In [12]:
df = pd.read_csv("./greek_training_data/greek_sentences.csv")

In [13]:
text_labels = ["anger", "anticipation", "disgust", "fear", "joy", "sadness", "surprise", "trust"]
sample_text = "ἐγώ ὑμᾶς ἐπαινῶ"

In [14]:
# run the model against the sample text
# model score outputs indicate a confidence rating that the model has applied to the sample text
classifier(sample_text, text_labels, multi_label=False)

{'sequence': 'ἐγώ ὑμᾶς ἐπαινῶ',
 'labels': ['anticipation',
  'trust',
  'joy',
  'surprise',
  'fear',
  'anger',
  'sadness',
  'disgust'],
 'scores': [0.515584409236908,
  0.16567815840244293,
  0.08153747767210007,
  0.0718863382935524,
  0.046777334064245224,
  0.04591739550232887,
  0.044985320419073105,
  0.02763357199728489]}

In [15]:
# run prediction on entire given corpus.
# text_column is the column of the dataframe that holds the text in question
# text_labels is the emotional labels that should be addressed

def predict_sentiment(df, text_column, text_labels):
    result_list = []
    for index, row in df.iterrows():
        sequence_to_classify = row[text_column]
        result = classifier(sequence_to_classify, text_labels, multi_label = False)
        result['sentiment'] = result['labels'][0]
        result['score'] = result['scores'][0]
        result_list.append(result)
    result_df = pd.DataFrame(result_list)[['sequence','sentiment', 'score']]
    result_df = pd.merge(df, result_df, left_on = "text", right_on="sequence", how = "left")
    return result_df

In [16]:
results_df = predict_sentiment(df, "text", text_labels)

In [17]:
print(results_df)

                                 text                           sequence  \
0                     ἐγώ ὑμᾶς ἐπαινῶ                    ἐγώ ὑμᾶς ἐπαινῶ   
1          ὁ στρατιώτης ἐδωκε χρήματα         ὁ στρατιώτης ἐδωκε χρήματα   
2                ἐγώ οἶδα τούς Ὅρκους               ἐγώ οἶδα τούς Ὅρκους   
3      ὁ ποιητής τόν στρατιώτην τιμᾷ.     ὁ ποιητής τόν στρατιώτην τιμᾷ.   
4   Οἱ κύνες τούς διώκοντας φεύγουσι.  Οἱ κύνες τούς διώκοντας φεύγουσι.   
5               ἐγραψα την ἐπιστολήν.              ἐγραψα την ἐπιστολήν.   
6                     Δῶρα ἠγαγόμην.                     Δῶρα ἠγαγόμην.    
7              Βίοτος πολλά διδάσκει.             Βίοτος πολλά διδάσκει.   
8                  ἡ γυνή ἐστί ἀγαθή.                 ἡ γυνή ἐστί ἀγαθή.   
9          ὁ ποιητής κάλλιστος ἐστὶν.         ὁ ποιητής κάλλιστος ἐστὶν.   
10          ἡ πόλις γίγνεται πλούσια.          ἡ πόλις γίγνεται πλούσια.   
11         το στράτευμα ἐφάνη πάμπολυ         το στράτευμα ἐφάνη πάμπολυ   
12   μεγάλα 